In [1]:
"""make variations of input image"""
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import argparse, os, sys, glob
import PIL
import torch
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange, repeat
from torchvision.utils import make_grid
from torch import autocast
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader,Dataset
from contextlib import nullcontext
import time
from pytorch_lightning import seed_everything

from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
from scripts.custom_Dataset import CustomDataset
from IPython.display import display, clear_output

In [2]:
class parser():
    def __init__(self):
        self.prompt = "a painting of a virus monster playing guitar"
        self.init_img = "outputs/img2img-samples/grid-0003.png" #path to the input image
        self.outdir = "../BLIP/experiment_1/stl_10/adv_test_generated_diffusion_trained_AA" #dir to write results to
        self.skip_grid = True
        self.skip_save = False
        self.ddim_steps = 50 #number of ddim sampling steps
        self.plms = False
        self.fixed_code = False
        self.ddim_eta = 0.0 #ddim eta (eta=0.0 corresponds to deterministic sampling
        self.n_iter = 1
        self.C = 4 #latent channels
        self.f = 8
        self.n_samples = 16 #batch size
        self.n_rows = 0
        self.scale = 5.0 #unconditional guidance scale: eps = eps(x, empty) + scale * (eps(x, cond) - eps(x, empty))
        self.strength = 0.5 #strength for noising/unnoising. 1.0 corresponds to full destruction of information in init image
        self.from_file = ""
        self.config = "configs/stable-diffusion/v1-inference.yaml"
        self.ckpt = "models/ldm/stable-diffusion-v1/model.ckpt"
        self.seed = 42
        self.precision = "autocast" #["full", "autocast"]
        

In [3]:
def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())


def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)
    model.cuda()
    model.eval()
    return model


def load_img(path):
    image = Image.open(path).convert("RGB")
    w, h = image.size
    print(f"loaded input image of size ({w}, {h}) from {path}")
    w, h = map(lambda x: x - x % 64, (w, h))  # resize to integer multiple of 32
    image = image.resize((w, h), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

In [4]:
opt = parser()
seed_everything(opt.seed)

config = OmegaConf.load(f"{opt.config}")
model = load_model_from_config(config, f"{opt.ckpt}")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

Global seed set to 42


Loading model from models/ldm/stable-diffusion-v1/model.ckpt
Global Step: 840000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.14.self_attn.k_proj.bias', 'vision_model.encoder.layers.0.layer_norm1.bias', 'vision_model.encoder.layers.13.self_attn.out_proj.bias', 'vision_model.encoder.layers.2.mlp.fc2.weight', 'vision_model.encoder.layers.22.self_attn.out_proj.bias', 'vision_model.encoder.layers.3.layer_norm2.bias', 'vision_model.encoder.layers.16.layer_norm1.weight', 'vision_model.encoder.layers.14.mlp.fc1.weight', 'vision_model.encoder.layers.23.self_attn.k_proj.weight', 'vision_model.encoder.layers.7.layer_norm2.bias', 'vision_model.encoder.layers.23.self_attn.out_proj.bias', 'vision_model.encoder.layers.3.self_attn.v_proj.weight', 'vision_model.encoder.layers.4.self_attn.q_proj.bias', 'vision_model.encoder.layers.3.mlp.fc2.weight', 'vision_model.encoder.layers.11.mlp.fc2.bias', 'vision_model.encoder.layers.17.self_attn.v_proj.weight', 'vision_model.encoder.layers.

In [5]:
#added by himanshu for adding batch processing
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
transform = transforms.Compose([transforms.Resize(size=(384,384)),
                                transforms.ToTensor(),
                                # transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                                ])
batch_size = opt.n_samples
num_classes = 10
image_folder = '../BLIP/experiment_1/stl_10/adv_test_diffusion_trained_AA/'
label_file = '../BLIP/experiment_1/stl_10/clean_test_labels.txt'
captions_file = "../BLIP/experiment_1/stl_10/adv_test_diffusion_trained_AA_captions.txt"
test_dataset = CustomDataset(image_folder, label_file, captions_file, transform=transform)
train_dataset = datasets.STL10(root="../data",
                           split="train",
                           transform=transform,
                          download=True)
# test_dataset = datasets.STL10(root="../data",
#                          split="test",
#                          transform=transform)

# train_loader = torch.utils.data.DataLoader(train_dataset,
#                                        batch_size=batch_size,
#                                        shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                     shuffle=False)

class_names = train_dataset.classes
# data_iterator = iter(train_loader)
# batch = next(data_iterator)

Files already downloaded and verified


In [8]:
if opt.plms:
    raise NotImplementedError("PLMS sampler not (yet) supported")
    sampler = PLMSSampler(model)
else:
    sampler = DDIMSampler(model)

os.makedirs(opt.outdir, exist_ok=True)
outpath = opt.outdir

batch_size = opt.n_samples
n_rows = opt.n_rows if opt.n_rows > 0 else batch_size

In [9]:
# data = [["This is a {}".format(class_names[i]) for i in batch[1]]]
# if not opt.from_file:
#     prompt = opt.prompt
#     assert prompt is not None
#     data = [batch_size * [prompt]]
# print(data, type(data))
# exit()

# else:
#     print(f"reading prompts from {opt.from_file}")
#     with open(opt.from_file, "r") as f:
#         data = f.read().splitlines()
#         data = list(chunk(data, batch_size))

sample_path = os.path.join(outpath, "samples")
os.makedirs(sample_path, exist_ok=True)
base_count = len(os.listdir(sample_path))
grid_count = len(os.listdir(outpath)) - 1

# assert os.path.isfile(opt.init_img)
# init_image = load_img(opt.init_img).to(device)
# init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)

# init_image = batch[0].to(device)
# print(init_image.shape, type(init_image))
# exit()
# init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space

# sampler.make_schedule(ddim_num_steps=opt.ddim_steps, ddim_eta=opt.ddim_eta, verbose=False)

assert 0. <= opt.strength <= 1., 'can only work with strength in [0.0, 1.0]'
t_enc = int(opt.strength * opt.ddim_steps)
print(f"target t_enc is {t_enc} steps")

precision_scope = autocast if opt.precision == "autocast" else nullcontext

target t_enc is 25 steps


In [10]:
with torch.no_grad():
    with precision_scope("cuda"):
        with model.ema_scope():
            tic = time.time()
            all_samples = list()
            for batch in tqdm(test_loader):
                clear_output(wait=True)
                data = [list(batch[2])]
                num = len(batch[0])
                init_image = batch[0].to(device)
                init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space
                sampler.make_schedule(ddim_num_steps=opt.ddim_steps, ddim_eta=opt.ddim_eta, verbose=False)
                for n in trange(opt.n_iter, desc="Sampling"):
                    for prompts in tqdm(data, desc="data"):
                        uc = None
                        if opt.scale != 1.0:
                            uc = model.get_learned_conditioning(num * [""])
                        if isinstance(prompts, tuple):
                            prompts = list(prompts)
                        c = model.get_learned_conditioning(prompts)

                        # encode (scaled latent)
                        z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*num).to(device))
                        # decode it
                        samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=opt.scale,
                                                 unconditional_conditioning=uc,)

                        x_samples = model.decode_first_stage(samples)
                        x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)

                        if not opt.skip_save:
                            for x_sample in x_samples:
                                x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                                Image.fromarray(x_sample.astype(np.uint8)).save(
                                    os.path.join(sample_path, f"{base_count}.png"))
                                base_count += 1
                        all_samples.append(x_samples)

                if not opt.skip_grid:
                    # additionally, save as grid
                    grid = torch.stack(all_samples, 0)
                    grid = rearrange(grid, 'n b c h w -> (n b) c h w')
                    grid = make_grid(grid, nrow=n_rows)

                    # to image
                    grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
                    Image.fromarray(grid.astype(np.uint8)).save(os.path.join(outpath, f'grid-{grid_count:04}.png'))
                    grid_count += 1

                toc = time.time()

print(f"Your samples are ready and waiting for you here: \n{outpath} \n"
      f" \nEnjoy.")

Running DDIM Sampling with 25 timesteps






















































Decoding image: 100%|██████████| 25/25 [00:07<00:00,  3.14it/s]

100%|██████████| 500/500 [1:42:49<00:00, 12.34s/it]

Your samples are ready and waiting for you here: 
../BLIP/experiment_1/stl_10/adv_test_generated_diffusion_trained_AA 
 
Enjoy.
